### Sign in with your Bentley account

In [ ]:
from evo.aio import AioTransport
from evo.common import APIConnector
from evo.oauth import AuthorizationCodeAuthorizer, EvoScopes, OAuthConnector

cache_location = "./notebook-data"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL
org_id = "<your-organization-id>"  # Replace with your organization ID
hub_url = "<your-hub-url>"  # Replace with your Evo hub URL

client_id = "daves-evo-client"  # Replace with your client ID
redirect_url = "http://localhost:32369/auth/callback"  # Replace with your redirect URL
org_id = "72748d4c-442f-4442-adf2-1d6dc8058f80"  # Replace with your organization ID
hub_url = "https://evo-demo.api.seequent.com"  # Replace with your Evo hub URL

client_name = "Evo Workspace API Examples"

transport = AioTransport(
    user_agent=client_name,
)

oauth_connector = OAuthConnector(
    transport=transport,
    client_id=client_id,
)

authorizer = AuthorizationCodeAuthorizer(
    oauth_connector=oauth_connector,
    redirect_url=redirect_url,
    scopes=EvoScopes.all_evo | EvoScopes.offline_access,
)

await authorizer.login()

# print(await authorizer.get_default_headers())

# print(authorizer.access_token)

### Create the APIConnector

In [ ]:
# Create the API connector to make direct API calls
connector = APIConnector(hub_url, transport, authorizer)

### Check if the user is an Evo admin

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/admin/orgs/{org_id}/workspaces"

    path_params = {
        "org_id": org_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    # Display as a table
    workspace_list = [
        {
            "Name": ws["display_name"],
            "ID": ws["id"],
            "User Role": ws["user_role"],
        }
        for ws in response_json["results"]
    ]

    df = pd.DataFrame(workspace_list)
    display(df)
except Exception as e:
    print(f"Error: User is not an Evo admin\n{e}")

### Add service account to the Evo instance

In [ ]:
import json

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/members"

    path_params = {
        "org_id": org_id,
    }

    body = {
        "users": [
            {
                "email": "service-W2X197A56C9bP2a5ruECohDeJ@apps.imsoidc.bentley.com",
                "roles": ["de402ff1-058e-4685-b4d4-9fe846270279"],
            }
        ]
    }

    api_response = await connector.call_api(
        method="POST",
        resource_path=resource_path,
        path_params=path_params,
        body=body,
        header_params={"Content-Type": "application/json"},
        response_types_map={
            "201": HTTPResponse,
        },
    )

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

except Exception as e:
    print(f"Error: Failed to add service identity\n{e}")

### Add service account to the Evo workspace

In [ ]:
import json

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

workspace_id = "<some-workspace-id>"

try:
    resource_path = "/workspace/admin/orgs/{org_id}/workspaces/{workspace_id}/users"

    # Replace with the actual user ID you want to assign
    user_id_to_assign = "<some-user-id>"
    email_to_assign = "<some-user-email>"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
    }

    body = {
        "user_id": user_id_to_assign,
        "email": email_to_assign,
        "role": "viewer",  # Can be "owner", "editor", or "viewer"
    }

    api_response = await connector.call_api(
        method="POST",
        resource_path=resource_path,
        path_params=path_params,
        header_params={"Content-Type": "application/json"},
        body=body,
        response_types_map={
            "201": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.CREATED:
        raise RuntimeError(f"Error: Failed to assign user role. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))
except Exception as e:
    print(f"Error assigning user role:\n{e}")